지난 글 [스트림 인터페이스](http://wikibootup.github.io/sicp/3-2-stream-interface.html)에서는 연산을 필요한 만큼만 계산하는 스트림의 개념을 살펴보았고,

스트림을 만들기 위한 기본 인터페이스를 만들어보았습니다. 

여기서는 만들었던 인터페이스를 이용해

1. __무한스트림__을 만들어보고,
2. 무한스트림 방식으로 팩토리얼 구하기 및 스트림의 부분합 구하기를 해보겠습니다.
3. 그리고, 근사값 구하기 문제를 스트림 가속기를 이용하여 풀어보겠습니다.

저번 글에서 `pair` 타입으로 `stream`을 만들었습니다.

쌍의 첫번째에는 현재 값이, 두번째에는 다음 값에 대한 '약속'이 들어있었습니다.

만약, 어떤 스트림에 __끝 없이 다음 값이 존재한다면__ 그 스트림을 무한스트림이라고 합니다.

'게으르지 않는' 프로그램에서는 한 번 명령하면 모든 연산을 한 방에 끝내려고 하기 떄문에,

'무한 재귀 함수' 같은 경우 `Maximum recursion error`와 같은 오류를 일으킵니다.

하지만 스트림 연산은 '한 번에 한 계산' 씩 수행하므로, 이런 오류를 피할 수 있습니다.

In [1]:
from modules.basic import cons_stream, apply, \
    nth_elements_extractor, display_stream, stream_enumerate_interval
from types import GeneratorType
from operator import add
import itertools

만약, 무한스트림의 값이 영원히 1이라면,

(1, 1, 1, ..., 1, ...)

아래와 같이 구현을 할 수 있습니다.

In [2]:
def ones():
    yield cons_stream(1, ones())

재귀 호출의 형태이므로 검증을 위해 처음 2개의 값만 확인해보면,

In [3]:
1 == next(ones())[0] == next(next(next(ones())[1]))[0]

True

무한 재귀 호출을 막기 위해 제너레이터를 과다 사용한 것 같아 아쉽긴 하지만

우선은 이 방법으로 무한스트림을 구현해보겠습니다.

깔끔한 연산을 위해서 제너레이터를 산출(`yield`)하는 함수를 추가로 만들겠습니다.

1. 제너레이터인지 확인한 후(`is_generator_contained`), 
2. 산출을 수행하는 두 함수( `yield_sequence`, `yield_stream` )을 정의합니다.


`yield_sequence`는 모든 차례열에 대하여,

`yield_stream`은 한 스트림 자체에 대하여 연산을 수행합니다.

In [4]:
from types import GeneratorType


def is_generator_contained(sequence):
    if isinstance(sequence, GeneratorType):
        return True
    
    if len(sequence) == 0:
        return False

    if isinstance(sequence[0], GeneratorType):
        return True
    else:
        return is_generator_contained(sequence[1:])

    
def yield_sequence(sequence):
    def operate(s):
        return tuple(map(lambda e:
            next(e) if isinstance(e, GeneratorType)
            else e, s))
    
    if is_generator_contained(sequence):
        return yield_sequence(operate(sequence))
    else:
        return sequence
    

def yield_stream(stream):
    if isinstance(stream, GeneratorType):
        return yield_stream(next(stream))
    else:
        return stream

이제 무한 스트림을 위한 `stream_map`을 만들어 보면( 위의 함수를 이용 ),

In [5]:
def stream_map(f, argstreams):
    def inner(f, argstreams):
        if not isinstance(argstreams[0], tuple):
            yield apply(f, argstreams)
        else:
            yield cons_stream(
                apply(f, nth_elements_extractor(0, argstreams)),
                stream_map(f, yield_sequence(
                            nth_elements_extractor(1, argstreams))))
    return inner(f, yield_sequence(argstreams))

검증을 위해 `ones()` 함수를 중복하여 `stream_map` 연산을 수행해보겠습니다.

적용할 기능은 덧셈( `add` )입니다.

In [6]:
yield_stream(stream_map(add, (ones(), ones())))

(2, <generator object promise at 0x104c9e3a8>)

In [7]:
yield_stream(stream_map(add, (ones(), ones(), ones(), ones())))

(4, <generator object promise at 0x104c9e558>)

다음으로, 유한 스트림을 넣어보겠습니다.

저번 글에서 풀었던 범위 내의 두 정수를 덧셈하는 연산을 해보면,

In [8]:
s2 = stream_enumerate_interval(5, 15)
s3 = stream_enumerate_interval(10, 20)

a = stream_map(add, (s2, s3))
display_stream(a)

15
17
19
21
23
25
27
29
31
33
35


유한스트림에서도 기능이 동작함을 확인하였습니다.

다음 번 글에 이어서 설명하도록 하겠습니다.